In [3]:
import pandas as pd
import numpy as np

file_path = "Public v4.1 K-12 School Shooting Database (8 28 2025) (2) 2.xlsx"

xls = pd.ExcelFile(file_path)
incident = pd.read_excel(xls, "Incident")
shooter = pd.read_excel(xls, "Shooter")
victim = pd.read_excel(xls, "Victim")
weapon = pd.read_excel(xls, "Weapon")

print("Shapes:")
print("Incident:", incident.shape)
print("Shooter:", shooter.shape)
print("Victim:", victim.shape)
print("Weapon:", weapon.shape)

Shapes:
Incident: (3136, 50)
Shooter: (3487, 8)
Victim: (3988, 6)
Weapon: (3168, 4)


In [13]:
# making copies so it doesn't mess with original
incident_df = incident.copy()
shooter_df = shooter.copy()
victim_df = victim.copy()
weapon_df = weapon.copy()

In [14]:
incident_df["Date"] = pd.to_datetime(incident_df["Date"], errors="coerce")

if "Year" in incident_df.columns:
    incident_df["Year"] = pd.to_numeric(incident_df["Year"], errors="coerce")
else:
    incident_df["Year"] = incident_df["Date"].dt.year

incident_df["Month"] = pd.to_numeric(incident_df["Month"], errors="coerce")
incident_df["Day"] = pd.to_numeric(incident_df["Day"], errors="coerce")

for col in ["Victims_Killed", "Victims_Wounded", "Number_Victims"]:
    if col in incident_df.columns:
        incident_df[col] = pd.to_numeric(incident_df[col], errors="coerce").fillna(0).astype(int)

incident_df["Total_Victims"] = incident_df["Victims_Killed"] + incident_df["Victims_Wounded"]


In [15]:
# 0: no injuries, 1: 1–2 victims, 2: 3–9 victims, 3: 10+ victims
def severity_level(n):
    if n == 0:
        return 0
    elif 1 <= n <= 2:
        return 1
    elif 3 <= n <= 9:
        return 2
    else:
        return 3

incident_df["Severity_Level"] = incident_df["Total_Victims"].apply(severity_level)

In [16]:
incident_df["Severe_Binary"] = (incident_df["Total_Victims"] >= 3).astype(int)

incident_df["Decade"] = (incident_df["Year"] // 10) * 10
incident_df["Decade"] = incident_df["Decade"].astype("Int64")

incident_df["Post_Columbine"] = (incident_df["Year"] >= 1999).astype(int)
incident_df["Post_SandyHook"] = (incident_df["Year"] >= 2012).astype(int)

# Standardize location text columns
for col in ["City", "State", "Location_Type", "Campus_Type"]:
    if col in incident_df.columns:
        incident_df[col] = incident_df[col].astype(str).str.strip()


In [18]:
def clean_yes_no(series):
    """
    Normalize Yes/No flags like 'Yes', 'No', 'Y', 'N', 'NO', etc.
    Returns boolean True/False, or NaN if unknown.
    """
    s = series.astype(str).str.strip().str.lower()
    return s.map({
        "yes": True,
        "y": True,
        "no": False,
        "n": False
    })

for col in ["Bullied", "Domestic_Violence", "Gang_Related", "Officer_Involved", "Active_Shooter_FBI"]:
    if col in incident_df.columns:
        incident_df[col + "_bool"] = clean_yes_no(incident_df[col])
    else:
        incident_df[col + "_bool"] = np.nan

def infer_motive(row):
    if row["Domestic_Violence_bool"] is True:
        return "Domestic violence"
    if row["Gang_Related_bool"] is True:
        return "Gang-related"
    if row["Bullied_bool"] is True:
        return "Bullying-related"
    if row["Officer_Involved_bool"] is True:
        return "Officer-involved"
    if row["Active_Shooter_FBI_bool"] is True:
        return "Active shooter"
    # Fallback
    return "Other/Unknown"

incident_df["Motive_Category"] = incident_df.apply(infer_motive, axis=1)



In [19]:
def shooter_age_group(x):
    # Already numeric
    if pd.api.types.is_number(x):
        try:
            x = float(x)
        except:
            return "Unknown"
        if pd.isna(x):
            return "Unknown"
        return "Juvenile" if x < 18 else "Adult"
    s = str(x).strip().lower()
    if s in ["adult", "a"]:
        return "Adult"
    if s in ["child", "juvenile", "teen"]:
        return "Juvenile"
    try:
        val = float(s)
        return "Juvenile" if val < 18 else "Adult"
    except:
        return "Unknown"

shooter_df["Age_Group"] = shooter_df["Age"].apply(shooter_age_group)

shooter_df["School_Affiliation"] = shooter_df["School_Affiliation"].astype(str).str.strip()

def aggregate_shooter(group):
    return pd.Series({
        "Shooter_Count": len(group),
        "Num_Juvenile_Shooters": (group["Age_Group"] == "Juvenile").sum(),
        "Num_Adult_Shooters": (group["Age_Group"] == "Adult").sum(),
        "Any_Shooter_Student": (group["School_Affiliation"] == "Student").any(),
        "Any_Shooter_Staff": (group["School_Affiliation"] == "Staff").any(),
        "Any_Shooter_Suicide": group["Shooter_Outcome"].astype(str).str.contains("Suicide", case=False, na=False).any()
    })

shooter_agg = shooter_df.groupby("Incident_ID").apply(aggregate_shooter).reset_index()


In [20]:
victim_df["Injury"] = victim_df["Injury"].astype(str).str.strip()
victim_df["School_Affiliation"] = victim_df["School_Affiliation"].astype(str).str.strip()

def aggregate_victim(group):
    return pd.Series({
        "Victim_Row_Count": len(group),
        "Victim_Fatalities_fromVictimTable": (group["Injury"] == "Fatal").sum(),
        "Victim_Wounded_fromVictimTable": (group["Injury"] == "Wounded").sum(),
        "Student_Victims": (group["School_Affiliation"] == "Student").sum(),
        "Staff_Victims": (group["School_Affiliation"] == "Staff").sum()
    })

victim_agg = victim_df.groupby("Incident_ID").apply(aggregate_victim).reset_index()


In [21]:
weapon_df["Weapon_Type"] = weapon_df["Weapon_Type"].astype(str).str.strip()

def aggregate_weapon(group):
    types = group["Weapon_Type"].dropna().unique().tolist()
    # Simple flags for common weapon categories
    return pd.Series({
        "Num_Weapons": len(group),
        "Weapon_Types_List": ", ".join(types),
        "Has_Rifle": any(group["Weapon_Type"].str.contains("Rifle", case=False, na=False)),
        "Has_Shotgun": any(group["Weapon_Type"].str.contains("Shotgun", case=False, na=False)),
        "Has_Handgun": any(group["Weapon_Type"].str.contains("Handgun", case=False, na=False))
    })

weapon_agg = weapon_df.groupby("Incident_ID").apply(aggregate_weapon).reset_index()


In [22]:
merged = (
    incident_df
    .merge(shooter_agg, on="Incident_ID", how="left")
    .merge(victim_agg, on="Incident_ID", how="left")
    .merge(weapon_agg, on="Incident_ID", how="left")
)

print("Merged shape:", merged.shape)

cols_of_interest = [
    "Incident_ID", "Date", "Year", "Decade",
    "City", "State", "Location_Type", "Campus_Type",
    "Total_Victims", "Victims_Killed", "Victims_Wounded",
    "Severity_Level", "Severe_Binary",
    "Motive_Category",
    "Shooter_Count", "Num_Juvenile_Shooters", "Num_Adult_Shooters",
    "Any_Shooter_Student", "Any_Shooter_Staff", "Any_Shooter_Suicide",
    "Num_Weapons", "Weapon_Types_List"
]

print(merged[cols_of_interest].head())


Merged shape: (3136, 78)
     Incident_ID       Date  Year  Decade         City State  \
0  20250827MNANM 2025-08-27  2025    2020  Minneapolis    MN   
1  20250826MNCRM 2025-08-26  2025    2020  Minneapolis    MN   
2  20250822ILMAP 2025-08-22  2025    2020       Peoria    IL   
3  20250822KYMAM 2025-08-22  2025    2020     Mayfield    KY   
4  20250820TNRIM 2025-08-20  2025    2020      Memphis    TN   

                Location_Type                     Campus_Type  Total_Victims  \
0         Both Inside/Outside     Multiple detached buildings             19   
1         Off School Property         Urban Street Standalone              7   
2  Outside on School Property     Multiple attached buildings              0   
3  Outside on School Property     Multiple attached buildings              1   
4                         nan  Single building indoor hallway              0   

   Victims_Killed  ...  Severe_Binary  Motive_Category  Shooter_Count  \
0               2  ...              

In [23]:
merged.to_csv("k12_school_shootings_merged_clean.csv", index=False)
incident_df.to_csv("k12_incident_clean.csv", index=False)
shooter_agg.to_csv("k12_shooter_agg.csv", index=False)
victim_agg.to_csv("k12_victim_agg.csv", index=False)
weapon_agg.to_csv("k12_weapon_agg.csv", index=False)

print("Cleaned files saved:")
print("- k12_school_shootings_merged_clean.csv")
print("- k12_incident_clean.csv")
print("- k12_shooter_agg.csv")
print("- k12_victim_agg.csv")
print("- k12_weapon_agg.csv")

Cleaned files saved:
- k12_school_shootings_merged_clean.csv
- k12_incident_clean.csv
- k12_shooter_agg.csv
- k12_victim_agg.csv
- k12_weapon_agg.csv
